## Modern convnet architecture patterns

### Modularity, hierarchy, and reuse

### Residual connections

**Residual block where the number of filters changes**

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation="relu")(inputs)
residual = x # 잔차 설정
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x) 
# 출력 파일의 수를 32개에서 64개로 늘림
# padding="same" : 패딩으로 인한 다운샘플링 방지 위함
residual = layers.Conv2D(64, 1)(residual) # 잔차의 필터를 64개로 만들어줌 
x = layers.add([x, residual]) # 이제 add할 수 있다. 

**Case where target block includes a max pooling layer**

In [2]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation="relu")(inputs)
residual = x
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(2, padding="same")(x)
residual = layers.Conv2D(64, 1, strides=2)(residual) # strides=2 -> maxpooling 과 downsampling 일치 위함
x = layers.add([x, residual])

In [3]:
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Rescaling(1./255)(inputs)

def residual_block(x, filters, pooling=False):
    residual = x
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
    if pooling:
        x = layers.MaxPooling2D(2, padding="same")(x)
        residual = layers.Conv2D(filters, 1, strides=2)(residual)
    elif filters != residual.shape[-1]:
        residual = layers.Conv2D(filters, 1)(residual)
    x = layers.add([x, residual])
    return x

x = residual_block(x, filters=32, pooling=True)
x = residual_block(x, filters=64, pooling=True)
x = residual_block(x, filters=128, pooling=False) # global average pooling을 사용하므로 false

x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 3)    0           ['input_3[0][0]']                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 32, 32, 32)   896         ['rescaling[0][0]']              
                                                                                                  
 conv2d_7 (Conv2D)              (None, 32, 32, 32)   9248        ['conv2d_6[0][0]']               
                                                                                              

### Batch normalization

In [ ]:
x = layers.Conv2D(32,3,use_bias=False)(x)
x = layers.BatchNormalization()(x) # 먼저 배치 정규화 후
x = layers,Activation("relu")[x] # activation 실행해야

배치 정규화 fine tuning시  
-> 정규화 layer는 untrainable로(내부 분산 및 평균이 계속 업데이트되어 튜닝에 방해가 될 수 있다) 

### Depthwise separable convolutions

### Putting it together: A mini Xception-like model

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yoonhanse","key":"3f017a293173dc2509f8acb74aa02bac"}'}

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats
!unzip -qq train.zip

 99% 800M/812M [00:04<00:00, 203MB/s]
100% 812M/812M [00:04<00:00, 203MB/s]
unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


In [6]:
!unzip -qq dogs-vs-cats.zip

In [7]:
!unzip -qq train.zip

- 모델은 보통 여러 convolution layers와 max pooling layer로 구성된 반복된 레이어블록으로 구성된다.
- 공간 피쳐맵의 크기가 줄어들면 레이어의 필터 수가 증가해야 한다.
- 넓고 얕은 것보다 좁고 깊은 것이 낫다.
- residual connection을 도입하면 더 깊은 네트워크를 훈련하는데 도움이 된다.
- convolution layer 뒤에 batch normalizaion 레이어를 도입하는 것이 유용하다.
- conv2d보다 separable conv2d layer가 더 유리할 수 있다. 

In [8]:
import os, shutil, pathlib
from tensorflow.keras.utils import image_dataset_from_directory

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [9]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
) # data augmentation 

In [10]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs) # data augmentation 

x = layers.Rescaling(1./255)(x) # scaling
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x) # conv2d 

for size in [32, 64, 128, 256, 512]: # residual connection 적용 
    residual = x

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    residual = layers.Conv2D(
        size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = layers.add([x, residual])

x = layers.GlobalAveragePooling2D()(x) # gloval average pooling 적용 -> feature을 1차원으로 만듦
x = layers.Dropout(0.5)(x) # 드롭아웃 적용 
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset)

Epoch 1/100
63/63 [==============================] - 44s 441ms/step - loss: 0.6952 - accuracy: 0.5605 - val_loss: 0.6928 - val_accuracy: 0.5050
Epoch 2/100
63/63 [==============================] - 27s 418ms/step - loss: 0.6553 - accuracy: 0.6175 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/100
63/63 [==============================] - 27s 421ms/step - loss: 0.6418 - accuracy: 0.6345 - val_loss: 0.6976 - val_accuracy: 0.5000
Epoch 4/100
63/63 [==============================] - 27s 421ms/step - loss: 0.6166 - accuracy: 0.6565 - val_loss: 0.7166 - val_accuracy: 0.5000
Epoch 5/100
63/63 [==============================] - 27s 415ms/step - loss: 0.6049 - accuracy: 0.6740 - val_loss: 0.8161 - val_accuracy: 0.5000
Epoch 6/100
63/63 [==============================] - 27s 424ms/step - loss: 0.5905 - accuracy: 0.6820 - val_loss: 0.7908 - val_accuracy: 0.5000
Epoch 7/100
63/63 [==============================] - 27s 424ms/step - loss: 0.5608 - accuracy: 0.7025 - val_loss: 0.8714 - val_accuracy: